# Check for sources

In [3]:
import sklearn as sk
import pandas as pd
import json

import functions as f
import fakenews_utilities as fns

import nltk
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()


with open("reference_verbs.json") as json_file:
    reference_list = json.load(json_file)

df = pd.read_csv("tweets_labeled.csv", engine='python') 
df.head() # this shows the head of the dataframe in the output

,tweet_id,text,label
0,1161040537207463936,'RT @SenJeffMerkley: The Endangered Species Ac...,1
1,1176360756239118342,'RT @LindseyGrahamSC: Interesting concept -- i...,1
2,1099036648573145088,'RT @RealJamesWoods: #BuildTheWall #DeportThem...,0
3,1092915693203480577,'RT @PatriotJackiB: Why would the MEXICAN GOV’...,0
4,1149038450668187654,'RT @TheOnion: Sweden Announces Plan To Get 10...,0


In [4]:
def clean(df):
    sentence = df['text'].str.replace(r'RT\s', '')
    sentence = sentence.str.replace(r'@\S*?\s', '')
    sentence = sentence.str.replace(r'@\S*?$', '')
    sentence = sentence.str.replace(r'https\S*?\s', ' ')  
    sentence = sentence.str.replace(r'https\S*?$', '')
    sentence = sentence.str.replace('\'', '')
    sentence = sentence.str.replace(r'…', '')
    sentence = sentence.str.replace(r'\s$', '')

    df['text'] = sentence.str.lower()
    return df

In [5]:
df_clean = clean(df)
df_clean.head()

df_new = df_clean[0:50]

In [6]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def get_POS_tags(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV,}
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w, get_POS_tags(w)) for w in nltk.word_tokenize(text)]

In [7]:
def add_ref_count(input_df): 
     
    text = input_df["text"]
    lem_text = text.apply(lemmatize_text)
    
    
    ret_list = [] 
    
    for line in lem_text:
        this_count = 0
   
        for ref_word in line: 

            if ref_word in reference_list :
                this_count += 1  

        ret_list.append(this_count)    
  
    return ret_list


ret_list = add_ref_count(df_new)
df_new.loc[:,'this_count'] = ret_list

/Users/shaniaspierings/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/shaniaspierings/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [9]:
df_new.head(50)

,tweet_id,text,label,this_count
0,1161040537207463936,the endangered species act saved the bald eagl...,1,0
1,1176360756239118342,"interesting concept -- impeach first, find fac...",1,2
2,1099036648573145088,#buildthewall #deportthemall,0,0
3,1092915693203480577,why would the mexican gov’t fund this? who are...,0,0
4,1149038450668187654,sweden announces plan to get 100% of energy fr...,0,0
5,1175456815674343424,warren: congress is complicit by failing to st...,1,0
6,1180809117310623744,a dozen current and former staff from state de...,1,2
7,1179840318935576578,this is a bombshell that isn’t getting much no...,1,1
8,1081722778125062144,planned parenthood erects billboards urging wo...,0,1
9,1158761795739217921,obstructing the prosperity of our country mus...,1,0


In [10]:
df_subset = df_new
corr = df_subset.corr()
corr

,tweet_id,label,this_count
tweet_id,1.000000,0.850326,0.300711
label,0.850326,1.000000,0.257153
this_count,0.300711,0.257153,1.000000
